In [1]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)


In [ ]:
# load env
from dotenv import load_dotenv
import os 
load_dotenv("../.env")

from tickapp.clients.signal_client import SignalClient

In [ ]:
# init client
client = SignalClient(phone_number=os.getenv("SIGNAL_PHONE_NUMBER"))

INFO:tickapp.clients.signal_client:✅ Signal Client initialisé pour +33695071416


In [ ]:
# receive messages
msgs = client.receive()
msgs

'{"envelope":{"source":"6aef6f35-7fcd-44c3-a6a4-e4c69f43535c","sourceNumber":null,"sourceUuid":"6aef6f35-7fcd-44c3-a6a4-e4c69f43535c","sourceName":"Carmelo Mileto","sourceDevice":1,"timestamp":1763934908289,"serverReceivedTimestamp":1763934909868,"serverDeliveredTimestamp":1763935344833,"dataMessage":{"timestamp":1763934908289,"message":"Ahahha","expiresInSeconds":0,"isExpirationUpdate":false,"viewOnce":false,"attachments":[{"contentType":"image/jpeg","filename":"signal-2025-11-23-225508.jpeg","id":"A5Cs_lKkx3xLZrJ_tOCU.jpeg","size":305032,"width":1152,"height":2048,"caption":null,"uploadTimestamp":1763934908429}],"groupInfo":{"groupId":"mPN0DQcs5cTJ+UMmJb9IDTGQMzCtPHP68fWQdZw/Uag=","groupName":"Tickets 🧾","revision":15,"type":"DELIVER"}}},"account":"+33695071416"}\n'

In [ ]:
# parse messages
msgs_p = client._parse_message(msgs)
msgs_p

Message(sender=Contact(number='6aef6f35-7fcd-44c3-a6a4-e4c69f43535c', name='Carmelo Mileto', uuid='6aef6f35-7fcd-44c3-a6a4-e4c69f43535c'), timestamp=datetime.datetime(2025, 11, 23, 22, 55, 8, 289000), text='Ahahha', attachments=[Attachment(id='A5Cs_lKkx3xLZrJ_tOCU.jpeg', content_type='image/jpeg', filename='signal-2025-11-23-225508.jpeg', size=305032, upload_timestamp_ms=1763934908429)], group=Group(id='mPN0DQcs5cTJ+UMmJb9IDTGQMzCtPHP68fWQdZw/Uag=', name='Unknown'), is_group_message=True, account='')

In [ ]:
# download attachment
path = client.download_attachment(
    phone_number=client.phone_number, 
    group_id=msgs_p.group.id, 
    attachment_id=msgs_p.attachments[0].id
)
path

PosixPath('/Users/carmelomileto/.local/share/signal-cli/attachments/A5Cs_lKkx3xLZrJ_tOCU.jpeg')